In [20]:
import difflib
import json
import IPython
import os
import pandas as pd
import pylcs
import sys

In [49]:
# set parameters 
model = 'StarCoder-1B'
language = 'python'     # ['python', 'java', 'csharp', 'typescript']
file = 'line_completion.jsonl'   #['line_completion.jsonl, 'line_completion_rg1_bm25.jsonl', 'line_completion_oracle_bm25.jsonl']

sort_asc = True  #Sort DataFrame ASC by default

whole_path_file = f"./data/{model}/{language}/result_{file}"

In [51]:
#get tabby_eval data from file to DataFrame
objs=[]
with open(whole_path_file) as f:
    for line in f:
        obj = json.loads(line)
        objs.append(obj['tabby_eval'])

df = pd.DataFrame(objs)

#filter df by unmatched prediction and groundtruth
tabby_eval_df = df[df['first_line_matched']==False].copy()

#add 'edit_distance' column to DataFrame
for index, row in tabby_eval_df.iterrows():
    lcs_sequence_len = pylcs.lcs_sequence_length(row['first_line_prediction'], row['first_line_groundtruth'])
    tabby_eval_df.loc[index, 'edit_distance'] = int((len(row['first_line_prediction'])-lcs_sequence_len) + (len(row['first_line_groundtruth'])-lcs_sequence_len))

#sort df by edit_distance
tabby_eval_df.sort_values('edit_distance',ascending=sort_asc)

,raw_prompt,prediction,first_line_prediction,first_line_groundtruth,first_line_matched,first_statement_prediction,first_statement_groundtruth,first_statement_matched,edit_distance
2568,"from classes import BaseModule, Response, MDEM...","create_html_table(DetailedResults['Accounts'],...","create_html_table(DetailedResults['Accounts'],...","list_to_html_table(linked_accounts_list, escap...",False,"create_html_table(DetailedResults['Accounts'],...","list_to_html_table(linked_accounts_list, escap...",False,476.0
471,import os\nimport time\n\nimport torch as th\n...,ModelType.ModelType.ModelType.ModelType.ModelT...,ModelType.ModelType.ModelType.ModelType.ModelT...,DEIR and not self.use_model_rnn:,False,ModelType.ModelType.ModelType.ModelType.ModelT...,DEIR and not self.use_model_rnn:,False,441.0
2406,import agents.utils.basicprompts as p\n\n\nsys...,"ask_question(question=system_init, prompt=""Wha...","ask_question(question=system_init, prompt=""Wha...",prompting_utils + p.using_steps + reevaluateAt...,False,"ask_question(question=system_init, prompt=""Wha...",prompting_utils + p.using_steps + reevaluateAt...,False,424.0
2332,import openai\nfrom .backend import Backend\n\...,"model_name, max_length=self.max_length, temper...","model_name, max_length=self.max_length, temper...",model_name)['data'][0]['embedding'],False,"model_name, max_length=self.max_length, temper...",model_name)['data'][0]['embedding'],False,385.0
807,\nimport os\nfrom argparse import ArgumentPars...,"num_joints, data_module.num_joints, data_modul...","num_joints, data_module.num_joints, data_modul...","skeleton, window, 1.0 / frequency,batch_size=b...",False,"num_joints, data_module.num_joints, data_modul...","skeleton, window, 1.0 / frequency,batch_size=b...",False,382.0
...,...,...,...,...,...,...,...,...,...
213,#! /usr/bin/env python\n# coding=utf8\n\nimpor...,add_GrpcServiceServicer_to_server(TestService(...,add_GrpcServiceServicer_to_server(TestService(...,add_GrpcServiceServicer_to_server(TestService(...,False,add_GrpcServiceServicer_to_server(TestService(...,add_GrpcServiceServicer_to_server(TestService(...,False,1.0
1978,"""""""A package for decoding and encoding each it...",get_buffer())\n self.file_desc.clos...,get_buffer()),getbuffer()),False,get_buffer()),getbuffer()),False,1.0
1319,import threading\nimport time\nfrom agency.pro...,process()\n\n self.__thread = threading...,process(),_process(),False,process(),_process(),False,1.0
826,import asyncio\nimport sys\nimport os\nsys.pat...,delete_old_resources_snapshots(time)\n\n ...,delete_old_resources_snapshots(time),delete_old_resource_snapshots(time),False,delete_old_resources_snapshots(time),delete_old_resource_snapshots(time),False,1.0


In [53]:
#set index of the DataFrame
index = 2336

#get prompt, first_line_prediction, first_line_groundtruth
prompt = df.iloc[index]['raw_prompt']
prediction = prompt.split("\n")[-1]+f"<end of prompt>"+df.iloc[index]['first_line_prediction']
groundtruth = prompt.split("\n")[-1]+f"<end of prompt>"+df.iloc[index]['first_line_groundtruth']
list_prediction = prediction.split('\n')
list_groundtruth = groundtruth.split('\n')

#show difference between first_line_prediction and first_line_groundtruth
prompt = prompt.replace("\n", "<br>")
html_file = difflib.HtmlDiff().make_file(list_prediction, list_groundtruth)
prompt_part = f"<h1>Prompt</h1><p>{prompt}</p>"
html_file = prompt_part + html_file
IPython.display.HTML(data=html_file)